In [1]:
import sys
sys.path.append('../src')
from data import get_documents_list, get_cleaned_dataframe_with_topics, get_cleaned_dataframe
from models import classification, get_embedding_model, get_BERTopic_model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


loaded spacy language model: en_core_web_sm


For monolingual execution, please change WIKI_LANGUAGES in config.ini file to language code of language you want to use

load the preprocessed texts in defined languages into a dataframe

In [2]:
df = get_cleaned_dataframe()
df.head()

,cleaned_texts,cleaned_paragraphs,cleaned_sentences,label
0,"[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septemb...","[[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septem...","[[Abel, Ehrlich, , Hebrew, אבל, ארליך, Septem...",0
1,"[Alexander, Zemlinsky, Alexander, von, Zemlins...","[[Alexander, Zemlinsky, Alexander, von, Zemlin...","[[Alexander, Zemlinsky, Alexander, von, Zemlin...",0
2,"[Alexander, Tansman, Polish, Aleksander, Tansm...","[[Alexander, Tansman, Polish, Aleksander, Tans...","[[Alexander, Tansman, Polish, Aleksander, Tans...",0
3,"[Alfred, Szendrei, also, Alfred, Sendrey, Alad...","[[Alfred, Szendrei, also, Alfred, Sendrey, Ala...","[[Alfred, Szendrei, also, Alfred, Sendrey, Ala...",0
4,"[Andre, Asriel, 22, February, 1922, 28, May, 2...","[[Andre, Asriel, 22, February, 1922, 28, May, ...","[[Andre, Asriel, 22, February, 1922, 28, May, ...",0


Decide what types of documents you want to train your embedding model on

In [3]:
documents = get_documents_list('cleaned_sentences')

[['Abel',
  'Ehrlich',
  ' ',
  'Hebrew',
  'אבל',
  'ארליך',
  'September',
  '3',
  '1915',
  'October',
  '30',
  '2003',
  'israeli',
  'composer'],
 [' 1997', 'Ehrlich', 'win', 'Israel', 'Prize', 'Music'],
 [' Erlich', 'bear', '1915', 'Cranz', 'East', 'Prussia'],
 [' 1934',
  'family',
  'flee',
  'Nazi',
  'Germany',
  'Yugoslavia',
  'pursue',
  'music',
  'study',
  'Zagreb'],
 [' leave',
  'Yugoslavia',
  '1939',
  'short',
  'stay',
  'Albania',
  'immigrate',
  'Mandatory',
  'Palestine',
  '\\n'],
 [' Israel',
  'continue',
  'study',
  'Eretz',
  'Israel',
  'Conservatory',
  'Jerusalem'],
 [' \\n',
  'teach',
  'various',
  'institute',
  'Israel',
  'Conservatory',
  'Rubin',
  'Academy',
  'Music',
  'Jerusalem',
  'Rubin',
  'Academy',
  'Music',
  'Tel',
  'Aviv',
  'Bar',
  'Ilan',
  'University',
  'Oranim',
  'Academic',
  'College'],
 [' see', 'list', 'music', 'student', 'teacher', 'c', 'F#Abel', 'Ehrlich'],
 [' \\n', 'die', 'October', '30', '2003', 'Tel', 'Aviv',

## Embedding Model Training
### Parameters:
- method = "muse", "fastText", or "Word2Vec"
- Optional: text_type = "cleaned_texts", "cleaned_texts_topics", ... (see more in embedding_models.py)
- Optional: training_type = "pretrained_aligned" or "from_scratch"

Used Configarations: 
- vector_size - vector size = 300
- window - num of words before and after the focus word considered as context for the word = 5
- min_count - the number of times a word must appear in our corpus in order to create a word vector. = 2

Train an embedding model with the train set

For multilingual fastText Model first train monolingual fastText models (use parameter method="fastText"). 
Then use muse library to align the embeddings: https://github.com/facebookresearch/MUSE
The command could look like this (supervised): 
``` shell
python supervised.py --src_lang en --tgt_lang de --src_emb /Users/jessicahassibi/Bachelor-Thesis/models/classification/FastText/FastText_en_cleaned_texts.vec --tgt_emb /Users/jessicahassibi/Bachelor-Thesis/models/classification/FastText/FastText_de_cleaned_texts.vec --n_refinement 5 --dico_train default --cuda=False
```
or like this (unsupervised):
``` shell
python unsupervised.py --src_lang en --tgt_lang de --src_emb /Users/jessicahassibi/Bachelor-Thesis/models/classification/FastText/FastText_en_cleaned_texts.vec --tgt_emb /Users/jessicahassibi/Bachelor-Thesis/models/classification/FastText/FastText_de_cleaned_texts.vec --n_refinement 5 --cuda=False
```

Place the aligned vectors to /models/classification/FastText/aligned_embeddings and use code in next cell with method="muse" to combine those embeddings and get multilingual word vectors.

In [4]:
model, word_vectors = get_embedding_model(documents, text_type="cleaned_sentences", method = "fastText", training_type="from_scratch")

fastText model not found.
Training new model on 4947 documents.
Saving new model.
Model:  /home/Bachelor-Thesis/models/classification/FastText/FastText_en_cleaned_sentences  loaded


In [5]:
# Similarity of vectors for a word 
word_vectors.similarity('composer', 'Komponist')

0.9999951